In [ ]:
!pip install PyMuPDF -q

import tensorflow as tf
import numpy as np
import pandas as pd
import fitz
import pickle
import re
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences
from google.colab import files

In [ ]:
class Attention(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                 initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1),
                                 initializer="zeros")
        super(Attention, self).build(input_shape)

    def call(self, x):
        e = tf.keras.backend.tanh(tf.keras.backend.dot(x, self.W) + self.b)
        a = tf.keras.backend.softmax(e, axis=1)
        return tf.keras.backend.sum(a * x, axis=1)

In [ ]:
print("📂 Vui lòng upload các file sau:")
print("1. word_difficulty_bilstm.h5")
print("2. char_vocab.pkl")
print("3. word_tokenizer.pkl")
print("4. scaler.pkl")
print("5. thresholds.pkl")
print("6. WordDifficulty.csv")

uploaded = files.upload()

📂 Vui lòng upload các file sau:
1. word_difficulty_final.h5
2. char_vocab.pkl
3. word_tokenizer.pkl
4. scaler.pkl
5. thresholds.pkl
6. WordDifficulty.csv


Saving WordDifficulty.csv to WordDifficulty.csv
Saving char_vocab.pkl to char_vocab.pkl
Saving scaler.pkl to scaler.pkl
Saving thresholds.pkl to thresholds.pkl
Saving word_difficulty_final.h5 to word_difficulty_final.h5
Saving word_tokenizer.pkl to word_tokenizer.pkl


In [ ]:
# 1. Load Model & Configs
print("⏳ Loading Model & Configs...")
model = tf.keras.models.load_model(
    "word_difficulty_bilstm.h5",
    custom_objects={"Attention": Attention},
    compile=False
)
with open("char_vocab.pkl", "rb") as f: char_vocab = pickle.load(f)
with open("word_tokenizer.pkl", "rb") as f: word_tokenizer = pickle.load(f)
with open("scaler.pkl", "rb") as f: scaler = pickle.load(f)
with open("thresholds.pkl", "rb") as f:
    threshold_data = pickle.load(f)
    th1, th2 = threshold_data["th1"], threshold_data["th2"]

# --- TẠO TỪ ĐIỂN TẦN SUẤT TỪ CSV ---
print("⏳ Đang tạo từ điển tần suất từ WordDifficulty.csv...")
try:
    df_ref = pd.read_csv("WordDifficulty.csv")
    # Làm sạch từ trong từ điển (chữ thường, bỏ nháy đơn)
    df_ref["clean_word"] = df_ref["Word"].str.lower().str.replace("'", "")

    # Tạo Dictionary: { 'word': frequency_value }
    freq_dict = dict(zip(df_ref["clean_word"], df_ref["Log_Freq_HAL"]))

    # Tính giá trị trung bình để gán cho các từ lạ
    avg_freq = df_ref["Log_Freq_HAL"].mean()
    print(f"✅ Đã load {len(freq_dict)} từ vào từ điển tần suất.")
    print(f"👉 Giá trị tần suất trung bình (dùng cho từ lạ): {avg_freq:.4f}")

except FileNotFoundError:
    print("❌ LỖI: Không tìm thấy file 'WordDifficulty.csv'. Vui lòng upload file này!")
    freq_dict = {}
    avg_freq = 0

⏳ Loading Model & Configs...
⏳ Đang tạo từ điển tần suất từ WordDifficulty.csv...
✅ Đã load 40004 từ vào từ điển tần suất.
👉 Giá trị tần suất trung bình (dùng cho từ lạ): 6.1632


In [ ]:
# --- 1. Xử lý văn bản ---
def clean_token(w):
    return str(w).lower().replace("'", "")

# --- 2. Đếm âm tiết ---
def count_syllables(word):
    word = str(word).lower()
    count = 0
    vowels = "aeiouy"
    if len(word) == 0: return 0
    if word[0] in vowels: count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e"): count -= 1
    if count == 0: count += 1
    return count

# --- 3. Trích xuất đặc trưng ---
def extract_manual_features(words):
    features = []
    vowels = set("aeiou")

    for w in words:
        w_clean = clean_token(w)

        length = len(w_clean)
        num_vowels = sum(1 for c in w_clean if c in vowels)
        syllables = count_syllables(w_clean)
        consonant_ratio = (length - num_vowels) / length if length > 0 else 0

        # --- TRA CỨU TẦN SUẤT ---
        if w_clean in freq_dict:
            freq = freq_dict[w_clean]  # Lấy giá trị thực tế từ dataset
        else:
            freq = avg_freq            # Từ lạ -> Dùng giá trị trung bình

        # Vector 5 chiều: [Độ dài, Nguyên âm, Âm tiết, Tỷ lệ phụ âm, Tần suất]
        features.append([length, num_vowels, syllables, consonant_ratio, freq])

    return np.array(features)

# --- 4. Mã hóa Ký tự ---
def words_to_char_seq(words, vocab, maxlen=21):
    seqs = [[vocab.get(c, 0) for c in w] for w in words]
    return pad_sequences(seqs, maxlen=maxlen, padding="post")

# --- 5. Phân loại ---
def classify_score(score, t1, t2):
    if score < t1: return 0
    elif score < t2: return 1
    else: return 2

In [ ]:
def predict_word_difficulty(words_list):
    # 1. Làm sạch danh sách từ
    clean_words = [clean_token(w) for w in words_list if len(clean_token(w)) > 0]

    if len(clean_words) == 0:
        return []

    # 2. Tạo Input 1: Character Sequence
    X_char = words_to_char_seq(clean_words, char_vocab, maxlen=21)

    # 3. Tạo Input 2: Word Indices
    X_word_seq = word_tokenizer.texts_to_sequences(clean_words)
    X_word = np.array([seq[0] if len(seq) > 0 else 0 for seq in X_word_seq])

    # 4. Tạo Input 3: Manual Features
    X_man = extract_manual_features(clean_words)
    X_man = scaler.transform(X_man)

    # 5. Dự đoán
    print("🤖 Đang dự đoán...")
    pred_scores = model.predict([X_char, X_word, X_man], verbose=0).flatten()

    # 6. Hậu xử lý kết quả
    results = []
    for w, score in zip(clean_words, pred_scores):
        level = classify_score(score, th1, th2)
        results.append({
            "Word": w,
            "Score": round(float(score), 5),
            "Level": level,
            "Label": ["Dễ", "Trung bình", "Khó"][level]
        })

    return pd.DataFrame(results)

In [ ]:
test_words = [
    "cat", "dog", "run",
    "beautiful", "university",
    "photosynthesis", "phenomenon", "extraordinary",
    "algorithm", "cryptography"
]

df_res = predict_word_difficulty(test_words)
print("\nKẾT QUẢ DỰ ĐOÁN:")
print(df_res)

🤖 Đang dự đoán...

KẾT QUẢ DỰ ĐOÁN:
             Word    Score  Level       Label
0             cat  0.10511      0          Dễ
1             dog  0.00154      0          Dễ
2             run -0.00443      0          Dễ
3       beautiful  0.51009      1  Trung bình
4      university  0.51420      1  Trung bình
5  photosynthesis  1.59437      2         Khó
6      phenomenon  0.89195      1  Trung bình
7   extraordinary  1.03244      1  Trung bình
8       algorithm  0.50298      1  Trung bình
9    cryptography  1.16305      1  Trung bình


In [ ]:
# Upload PDF
uploaded_pdf = files.upload()
pdf_path = list(uploaded_pdf.keys())[0]

# Đọc text
print(f"📄 Đang đọc file: {pdf_path}")
text = ""
with fitz.open(pdf_path) as doc:
    for page in doc:
        text += page.get_text()

# Tách từ
all_words = text.split()
print(f"Tìm thấy {len(all_words)} từ. Đang xử lý...")

# Dự đoán
df_final = predict_word_difficulty(all_words)

# Hiển thị thống kê
print("\n--- THỐNG KÊ ---")
print(df_final["Label"].value_counts())

# Lưu file CSV
csv_name = "pdf_difficulty_results.csv"
df_final.to_csv(csv_name, index=False)
print(f"\n💾 Đã lưu kết quả vào: {csv_name}")
files.download(csv_name)

# Hiển thị 20 dòng đầu
df_final.head(20)

Saving srs_template.pdf to srs_template.pdf
📄 Đang đọc file: srs_template.pdf
Tìm thấy 2024 từ. Đang xử lý...
🤖 Đang dự đoán...

--- THỐNG KÊ ---
Label
Dễ            1544
Trung bình     312
Khó             34
Name: count, dtype: int64

💾 Đã lưu kết quả vào: pdf_difficulty_results.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Word,Score,Level,Label
0,copyright,0.41500,0,Dễ
1,by,0.07250,0,Dễ
2,karl,0.59751,1,Trung bình
3,e,1.66703,2,Khó
4,wiegers,0.59803,1,Trung bình
5,permission,0.27713,0,Dễ
6,is,-0.18760,0,Dễ
7,granted,0.02877,0,Dễ
8,to,-0.12110,0,Dễ
9,use,0.02912,0,Dễ
